In [1]:
# !pip install seaborn


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import scipy.stats as stats

In [3]:
# Set up files
dir_firstSess = 'data/corr_firstSess'
files_firstSess = [f for f in os.listdir(dir_firstSess) if f.endswith('.csv')]

dir_secondSess = 'data/corr_secondSess'
files_secondSess = [f for f in os.listdir(dir_secondSess) if f.endswith('.csv')]

numTotalSessions = len(files_firstSess) + len(files_secondSess)

print('Number of total sessions: ', numTotalSessions)

Number of total sessions:  316


# Data cleaning

## Check if all experiments are completed

In [4]:
def checkIfComplete(mydir, files):
    
    numSess = len(files)
    
    for sess in range(numSess):

        # Read the CSV file
        file_path = os.path.join(mydir, files[sess])
        mainOutput = pd.read_csv(file_path)

        # Exclusion criteria 1: experiment completed
        # print(files[sess])
        complete_val = mainOutput['experimentCompleteBool'].dropna().iloc[0]
        complete_bool = str(complete_val) == 'True'
        if not complete_bool:
            prolificID = mainOutput['ProlificParticipantID'].dropna().iloc[0]
            print(f'Warning: incomplete experiment (session: {sess})')
            
    print(f'Completion check completed! ({numSess} files total)')
                        

In [5]:
checkIfComplete(dir_firstSess, files_firstSess)

Completion check completed! (167 files total)


In [6]:
checkIfComplete(dir_secondSess, files_secondSess)

Completion check completed! (149 files total)


## Acquire thresholds for all tasks

### Functions:

In [7]:
def getThresholds(mydir, files, condition_names, num_trials_per_staircase=35,
                     exclude_trial_count_bool=True, exclude_questSD=True):
    
    all_data = []
    
    numSess = len(files)
    numThresholdsCat = len(condition_names)
    
    for sess in range(numSess):

        # Read the CSV file
        file_path = os.path.join(mydir, files[sess])
        mainOutput = pd.read_csv(file_path)
        subj_logThresholds = {}

        # print(files[sess])
        # print(mainOutput['ProlificParticipantID'].dropna())
        prolificID = mainOutput['ProlificParticipantID'].dropna().iloc[0]
        subj_logThresholds['prolificID'] = prolificID
                
        for cat in range(numThresholdsCat):
            
            condition_name = condition_names[cat]
                        
            condition_data = mainOutput[mainOutput['conditionName'] == condition_name]

            assert(len(condition_data.questMeanAtEndOfTrialsLoop.dropna()) == 1)
            subj_logThresholds[condition_name] = condition_data.questMeanAtEndOfTrialsLoop.dropna().iloc[0]

            if exclude_trial_count_bool: 
                # Count trials sent to quest
                trial_sent = condition_data['trialGivenToQuest']           
                num_trial_sent = sum(str(this_trial) == 'True' for this_trial in trial_sent)
                num_trial_not_sent = sum(str(this_trial) == 'False' for this_trial in trial_sent)
                trial_sent_bool = num_trial_sent >= num_trials_per_staircase
                num_missing_line = sum(trial_sent.isna())
                if not trial_sent_bool:
                    subj_logThresholds[condition_name] = np.nan
                    # print(files[sess])
                    # print(f'Warning1: not enough trials (Session {sess}, condition {condition_name})')
                    # print(f'Num total trials: {len(trial_sent) - 1}')
                    # print(f'Num trials missing: {num_trials_per_staircase - num_trial_sent}')
                    # print(f'Num trials marked as not sent: {num_trial_not_sent}')
                    # print(f'Num lines missing: {num_missing_line - 1}')

            if exclude_questSD:
                questSD = condition_data['questSDAtEndOfTrialsLoop'].dropna().iloc[0]
                small_questSD_bool = questSD < 0.1
                if not small_questSD_bool:
                    subj_logThresholds[condition_name] = np.nan
                    # print(f'Warning2: large SD (Session {sess}, condition {condition_name}, SD = {questSD})')
        
        all_data.append(subj_logThresholds)
        
        all_data_df = pd.DataFrame(all_data)
        
    return all_data_df
            


In [8]:
def getRSVPThresholds(mydir, files, condition_names, num_trials_per_staircase=24,
                     exclude_trial_count_bool=True, exclude_questSD=True):
    
    all_data = []
    
    numSess = len(files)
    numThresholdsCat = len(condition_names)
    
    for sess in range(numSess):

        file_path = os.path.join(mydir, files[sess])
        mainOutput = pd.read_csv(file_path)
        subj_wpm = {}
                
        prolificID = mainOutput['ProlificParticipantID'].dropna().iloc[0]
        subj_wpm['prolificID'] = prolificID
                
        for cat in range(numThresholdsCat):
            
            condition_name = condition_names[cat]
                        
            condition_data = mainOutput[mainOutput['conditionName'] == condition_name]

            # Extract threshold: 
            # check that only 1 threshold is reported for this condition
            assert(len(condition_data.questMeanAtEndOfTrialsLoop.dropna()) == 1)
            thresholds_raw_log = condition_data.questMeanAtEndOfTrialsLoop.dropna().iloc[0]
            subj_wpm[condition_name] = np.power(10, np.log10(60) - thresholds_raw_log)

            if exclude_trial_count_bool: 
                        
                # Count trials sent to quest
                trial_sent = condition_data['trialGivenToQuest']           
                num_trial_sent = sum(str(this_trial) == 'True' for this_trial in trial_sent)
                num_trial_not_sent = sum(str(this_trial) == 'False' for this_trial in trial_sent)
                trial_sent_bool = num_trial_sent >= num_trials_per_staircase
                num_missing_line = sum(trial_sent.isna())

                if not trial_sent_bool:
                    subj_wpm[condition_name] = np.nan
                    # print(files[sess])
                    # print(f'Warning1: not enough trials (Session {sess}, condition {condition_name})')
                    # print(f'Num total trials: {len(trial_sent) - 1}')
                    # print(f'Num trials missing: {num_trials_per_staircase - num_trial_sent}')
                    # print(f'Num trials marked as not sent: {num_trial_not_sent}')
                    # print(f'Num lines missing: {num_missing_line - 1}')

            if exclude_questSD:
                questSD = condition_data['questSDAtEndOfTrialsLoop'].dropna().iloc[0]
                small_questSD_bool = questSD < 0.1
                
                if not small_questSD_bool:
                    subj_wpm[condition_name] = np.nan
                    # print(f'Warning2: large SD (Session {sess}, condition {condition_name}, SD = {questSD})')
            
        
        all_data.append(subj_wpm)
        
        all_data_df = pd.DataFrame(all_data)
        
    return all_data_df
        

In [9]:
def getOrdReadingSpeed(mydir, files, condition_dict,
                       accuracy_criterion_percent=60, wpm_criterion=400):
    '''
    accuracy_criterion_percent: the reading speed will be marked as np.nan if the accuracy for the comprehension question
                                is lower than this percentage
    wpm_criteiron: the reading speed will be marked as np.nan if it is higher than this percentage
    '''
    
    condition_names = list(condition_dict.keys())
    
    all_data = []
    
    numSess = len(files)
    numThresholdsCat = len(condition_names)
    
    for sess in range(numSess):
        
        # Read the CSV file
        file_path = os.path.join(mydir, files[sess])
        mainOutput = pd.read_csv(file_path)
        subj_wpm = {}
                
        prolificID = mainOutput['ProlificParticipantID'].dropna().iloc[0]
        subj_wpm['prolificID'] = prolificID
        
        for cat in range(numThresholdsCat):
            
            condition_name = condition_names[cat]

            # Check if the participant answered 3 or more questions correctly
            question_labels = condition_dict[condition_name]
            num_questions = len(question_labels)
            question_correct_bool = np.full(num_questions,np.nan)
            for qq in range(num_questions):

                qq_data = mainOutput[mainOutput['questionAndAnswerNickname'] == question_labels[qq]]
                
                question_correct_bool[qq] = (qq_data['questionAndAnswerCorrectAnswer'].item() == qq_data['questionAndAnswerResponse'].item())
                
            percent_correct = sum(question_correct_bool) / num_questions * 100 
            
            # calculate reading speed
            speed_data = mainOutput[mainOutput['conditionName'] == condition_name]
            numWords = speed_data['readingPageWords'].dropna()
            reading_time = speed_data['readingPageDurationOnsetToOffsetSec'].dropna()
            pg_wordsPerMin = numWords / (reading_time / 60)
            include_wordsPerMin = pg_wordsPerMin[1:len(pg_wordsPerMin)-1] # exclude first and last page
            subj_wpm[condition_name] = np.mean(include_wordsPerMin)
            
            if percent_correct < accuracy_criterion_percent: 
                # print(f'Warning: percent correct is too low: session {sess}, passage {cat}')
                subj_wpm[condition_name] = np.nan

            if np.mean(include_wordsPerMin) > wpm_criterion:
                # print(f'Warning: ordinary reading speed is too high: session {sess}, passage {cat}')
                # print(f'-- wpm: {np.mean(include_wordsPerMin)}')
                subj_wpm[condition_name] = np.nan

    
                
        all_data.append(subj_wpm)
        
        all_data_df = pd.DataFrame(all_data)
        
    return all_data_df
            

### Acquire thresholds:

- Note that data acquired for RSVP and Ordinary reading are in words per min -- not logged

- All the other thresholds are logged

In [10]:
# first session

thresholds_names_sess1 = ['crowding_R8_block1','crowding_L8_block1',
                          'crowding_R8_block2','crowding_L8_block2',
                          'acuity_R8_block1','acuity_L8_block1']
df_firstSess = getThresholds(dir_firstSess, files_firstSess, thresholds_names_sess1, exclude_trial_count_bool=True, exclude_questSD=True)

thresholds_vernier_sess1 = ['vernier_R8_block1','vernier_L8_block1']
df_firstSess_vernier = getThresholds(dir_firstSess, files_firstSess, thresholds_vernier_sess1, exclude_trial_count_bool=True, exclude_questSD=False)

thresholds_rsvp_sess1 = ['rsvp_foveal_block1']
df_firstSess_rsvp = getRSVPThresholds(dir_firstSess, files_firstSess, thresholds_rsvp_sess1, exclude_trial_count_bool=True, exclude_questSD=True)

thresholds_names_read1 = {
        'reading_Beaver_block1': ['Beaver_1','Beaver_2','Beaver_3','Beaver_4','Beaver_5'],
        'reading_Winter_block2': ['Winter_1','Winter_2','Winter_3','Winter_4','Winter_5']}
df_firstSess_reading = getOrdReadingSpeed(dir_firstSess, files_firstSess, thresholds_names_read1, accuracy_criterion_percent=60, wpm_criterion=400)

In [11]:
# second session

thresholds_names_sess2 = ['crowding_R8_block3','crowding_L8_block3',
                          'crowding_R8_block4','crowding_L8_block4',
                          'acuity_R8_block2','acuity_L8_block2']
df_secondSess = getThresholds(dir_secondSess, files_secondSess, thresholds_names_sess2, exclude_trial_count_bool=True, exclude_questSD=True)

thresholds_vernier_sess2 = ['vernier_R8_block2','vernier_L8_block2']
df_secondSess_vernier = getThresholds(dir_secondSess, files_secondSess, thresholds_vernier_sess2, exclude_trial_count_bool=True, exclude_questSD=False)

thresholds_rsvp_sess2 = ['rsvp_foveal_block2']
df_secondSess_rsvp = getRSVPThresholds(dir_secondSess, files_secondSess, thresholds_rsvp_sess2, exclude_trial_count_bool=True, exclude_questSD=True)

thresholds_names_read2 = {
        'reading_Desert_block1': ['Desert_1','Desert_2','Desert_3','Desert_4','Desert_5'],
        'reading_Islands_block2': ['Islands_1','Islands_2','Islands_3','Islands_4','Islands_5']}

df_secondSess_reading = getOrdReadingSpeed(dir_secondSess, files_secondSess, thresholds_names_read2, accuracy_criterion_percent=60, wpm_criterion=400)


In [12]:
# merge data frames
df_first_merge = pd.merge(df_firstSess, df_firstSess_vernier, on="prolificID", how="inner")
df_first_merge = pd.merge(df_first_merge, df_firstSess_rsvp, on="prolificID", how="inner")
df_first = pd.merge(df_first_merge, df_firstSess_reading, on="prolificID", how="inner")

df_second_merge = pd.merge(df_secondSess, df_secondSess_vernier, on="prolificID", how="inner")
df_second_merge = pd.merge(df_second_merge, df_secondSess_rsvp, on="prolificID", how="inner")
df_second = pd.merge(df_second_merge, df_secondSess_reading, on="prolificID", how="inner")


df_both_sessions = pd.merge(df_first, df_second, on="prolificID", how="inner")
# display(df_both_sessions)
# print(df_both_sessions.columns)

### For each task, exclude all thresholds from a participant if one threshold is abnormal

In [13]:
def nan_out_task(df, task_prefixes):
    df = df.copy()
    for prefix in task_prefixes:
        task_cols = [col for col in df.columns if col.startswith(prefix)]
        # Find rows where any of the task columns is nan
        mask = df[task_cols].isna().any(axis=1)
        # Set all task columns to nan for those rows
        df.loc[mask, task_cols] = np.nan
    return df

In [ ]:
task_prefixes = ['crowding', 'vernier', 'acuity', 'rsvp', 'reading']

# Apply to your data frame
df_both_sessions_exclude_subj_task = nan_out_task(df_both_sessions, task_prefixes)

### Average left and right thresholds for acuity and crowding

In [ ]:
df_both_sessions_averagedRL = df_both_sessions_exclude_subj_task.copy()

# Average left and right thresholds for acuity, crowding, and vernier in each block
blocks = ['block1', 'block2', 'block3', 'block4']
tasks = ['crowding', 'acuity', 'vernier']

for task in tasks:
    for block in blocks:
        col_R = f"{task}_R8_{block}"
        col_L = f"{task}_L8_{block}"
        avg_col = f"{task}_avg_{block}"
        if col_R in df_both_sessions_averagedRL.columns and col_L in df_both_sessions_averagedRL.columns:
            df_both_sessions_averagedRL[avg_col] = (
                df_both_sessions_averagedRL[[col_R, col_L]].mean(axis=1)
            )

# Now df_both_sessions_averagedRL contains averaged columns like 'crowding_avg_block1', etc.
# You can select only the averaged columns and prolificID for your new DataFrame:
avg_cols = ['prolificID'] + [f"{task}_avg_{block}" 
                             for task in tasks 
                             for block in blocks 
                             if f"{task}_avg_{block}" in df_both_sessions_averagedRL.columns]

df_averaged_thresholds = df_both_sessions_averagedRL[avg_cols].copy()



In [30]:
# Identify all RSVP and reading columns
rsvp_cols = [col for col in df_both_sessions_averagedRL.columns if col.startswith('rsvp')]
reading_cols = [col for col in df_both_sessions_averagedRL.columns if col.startswith('reading')]

# Combine with the averaged columns and prolificID
all_cols = ['prolificID'] + \
           [col for col in df_averaged_thresholds.columns if col != 'prolificID'] + \
           rsvp_cols + reading_cols

# Remove duplicates while preserving order
from collections import OrderedDict
all_cols = list(OrderedDict.fromkeys(all_cols))

# Create new DataFrame with all desired columns
df_averaged_thresholds_with_rsvp_reading = df_both_sessions_averagedRL[all_cols].copy()

print(df_averaged_thresholds_with_rsvp_reading.columns)

Index(['prolificID', 'crowding_avg_block1', 'crowding_avg_block2',
       'crowding_avg_block3', 'crowding_avg_block4', 'acuity_avg_block1',
       'acuity_avg_block2', 'vernier_avg_block1', 'vernier_avg_block2',
       'rsvp_foveal_block1', 'rsvp_foveal_block2', 'reading_Beaver_block1',
       'reading_Winter_block2', 'reading_Desert_block1',
       'reading_Islands_block2'],
      dtype='object')


In [29]:
print(df_averaged_thresholds.columns)

Index(['prolificID', 'crowding_avg_block1', 'crowding_avg_block2',
       'crowding_avg_block3', 'crowding_avg_block4', 'acuity_avg_block1',
       'acuity_avg_block2', 'vernier_avg_block1', 'vernier_avg_block2'],
      dtype='object')


# Analysis
For df_both_sessions and df_both_sessions_exclude_subj_task:
- letter, vernier, crowding are in logged deg: when we take the mean directly, it's taking the geometric mean
- rsvp and ordinary reading speeds are in words per minute: remember to log before taking the mean! 

In [25]:
df_for_analysis = df_both_sessions_exclude_subj_task.copy()

### Test-retest

In [22]:
df_for_analysis['crowding_R8_12'] = (df_for_analysis['crowding_R8_block1'] + 
                                      df_for_analysis['crowding_R8_block2']) / 2

df_for_analysis['crowding_L8_12'] = (df_for_analysis['crowding_L8_block1'] + 
                                      df_for_analysis['crowding_L8_block2']) / 2

df_for_analysis['crowding_R8_34'] = (df_for_analysis['crowding_R8_block3'] + 
                                      df_for_analysis['crowding_R8_block4']) / 2

df_for_analysis['crowding_L8_34'] = (df_for_analysis['crowding_L8_block3'] + 
                                      df_for_analysis['crowding_L8_block4']) / 2

In [23]:
df_for_analysis['ordreading_sess1'] = 10 ** ((np.log10(df_for_analysis['reading_Beaver_block1']) + 
                                               np.log10(df_for_analysis['reading_Winter_block2'])) / 2)

df_for_analysis['ordreading_sess2'] = 10 ** ((np.log10(df_for_analysis['reading_Desert_block1']) + 
                                               np.log10(df_for_analysis['reading_Islands_block2'])) / 2)

In [26]:
print(df_for_analysis.columns)


Index(['prolificID', 'crowding_R8_block1', 'crowding_L8_block1',
       'crowding_R8_block2', 'crowding_L8_block2', 'acuity_R8_block1',
       'acuity_L8_block1', 'vernier_R8_block1', 'vernier_L8_block1',
       'rsvp_foveal_block1', 'reading_Beaver_block1', 'reading_Winter_block2',
       'crowding_R8_block3', 'crowding_L8_block3', 'crowding_R8_block4',
       'crowding_L8_block4', 'acuity_R8_block2', 'acuity_L8_block2',
       'vernier_R8_block2', 'vernier_L8_block2', 'rsvp_foveal_block2',
       'reading_Desert_block1', 'reading_Islands_block2'],
      dtype='object')
